# 1. Import packages

In [1]:
import re
import string
from typing import Final, List

from IPython.core.display import display_html, HTML

from textattack import Attack, AttackArgs, Attacker
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions.classification import UntargetedClassification
from textattack.loggers import CSVLogger
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attacked_text import AttackedText
from textattack.transformations import WordSwap

/home/sypark/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Load model and dataset

In [2]:
# Load from my_model.py and my_dataset.py
from my_model import get_model
from my_dataset import dataset

model = get_model("cuda:2")

# 3. Define `CustomWordSwapEmbedding` class

In [3]:
class CustomWordSwapEmbedding(WordSwap):
    def __init__(self) -> None:
        hexword_regex: Final[re.Pattern] = re.compile(r"^[0-9a-fA-F]{4}$")
        self.replacement_words: List[str] = []

        with open("./models/encryptd_vocab.txt") as vocab_file:
            for vocab in vocab_file:
                self.replacement_words.append(
                    vocab.rstrip()
                ) if hexword_regex.fullmatch(vocab.rstrip()) else None
                continue
            pass

        return super().__init__(string.hexdigits)

    def _get_transformations(
        self, current_text: AttackedText, indices_to_modify: List[int]
    ) -> List[AttackedText]:
        words = current_text.words
        transformed_texts: List[AttackedText] = []

        for i in indices_to_modify:
            # if i < 22:
            #     continue
            
            word_to_replace = words[i]
            replacement_words = self._get_replacement_words(word_to_replace)
            transformed_texts_idx: List[AttackedText] = []

            for r in replacement_words:
                if r == word_to_replace:
                    continue

                indices: List[int] = []
                new_words: List[str] = []

                indices.append(i)
                new_words.append(r)

                if i < len(words) - 1:
                    post_r = r[2:] + words[i + 1][2:]
                    indices.append(i + 1)
                    new_words.append(post_r)
                    pass

                transformed_texts_idx.append(
                    current_text.replace_words_at_indices(indices, new_words)
                )
                continue

            transformed_texts.extend(transformed_texts_idx)
            continue

        return transformed_texts

    def _get_replacement_words(self, word: str) -> List[str]:
        replacement_words = [w for w in self.replacement_words if w.startswith(word[:2])]

        if word in replacement_words:
            replacement_words.remove(word)
            pass
        
        return replacement_words

    pass

# 4. Construct `textattack.Attack` object

In [4]:
# Construct the four fundamental components of the attack
goal_function = UntargetedClassification(model)
constraints = [
    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.9),
]
transformation = CustomWordSwapEmbedding()
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'my_model.Classifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


# 5. Construct `textattack.Attacker` object

In [5]:
# Attack until 1000 successful attacks are reached
attack_args = AttackArgs(num_successful_examples=100, random_seed=42)
attacker = Attacker(attack, dataset, attack_args)
attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CustomWordSwapEmbedding
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  1%|          | 1/100 [00:14<23:21, 14.16s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:14<23:57, 14.52s/it]

[[17 (100%)]] --> [[8 (93%)]]

e240 [[4067]] [[67c0]] c032 32e9 e967 67bf bfe4 e4f4 f480 [[8018]] [[1821]] 21f0 f077 77a0 [[a000]] [[0000]] 0001 0101 0108 [[080a]] [[0a06]] 06c5 c561 61bc bcf5 f573 73aa aada da01 0100 0059 597d 7d12 1266 667b [[7b40]] [[4053]] 5306 [[0647]] [[4745]] 451e 1e65 654c 4c05 056a 6a50 5074 [[744a]] [[4a7e]] 7e79 7901 0100 0000 0000 0000 0000 0000 0000 0000 0000 004a 4a9c 9cc1

e240 [[405b]] [[5bc0]] c032 32e9 e967 67bf bfe4 e4f4 f480 [[80d7]] [[d721]] 21f0 f077 77a0 [[a01b]] [[1b00]] 0001 0101 0108 [[0827]] [[2706]] 06c5 c561 61bc bcf5 f573 73aa aada da01 0100 0059 597d 7d12 1266 667b [[7bec]] [[ec53]] 5306 [[0671]] [[7145]] 451e 1e65 654c 4c05 056a 6a50 5074 [[7452]] [[527e]] 7e79 7901 0100 0000 0000 0000 0000 0000 0000 0000 0000 004a 4a9c 9cc1




...

--------------------------------------------- Result 98 ---------------------------------------------
[[8 (100%)]] --> [[5 (100%)]]

2571 71d4 d4ef ef03 033e 3ecd cd39 3920 2032 3280 8018 1821 21f0 f0e1 e1aa aa00 0000 0001 0101 0108 080a [[0a0f]] [[0f41]] 418c 8c91 91fd fde2 e2b6 b69b 9b01 0100 0000 0001 0101 0138 3800 0000 0002 0203 0364 6465 6566 6600 0000 0000 0011 1140 4040 4076 7665 6572 7273 7369 696f 6f6e 6e5f 5f63 636f 6f6d 6d6d 6d65 656e 6e74 7411

2571 71d4 d4ef ef03 033e 3ecd cd39 3920 2032 3280 8018 1821 21f0 f0e1 e1aa aa00 0000 0001 0101 0108 080a [[0a05]] [[0541]] 418c 8c91 91fd fde2 e2b6 b69b 9b01 0100 0000 0001 0101 0138 3800 0000 0002 0203 0364 6465 6566 6600 0000 0000 0011 1140 4040 4076 7665 6572 7273 7369 696f 6f6e 6e5f 5f63 636f 6f6d 6d6d 6d65 656e 6e74 7411




[Succeeded / Failed / Skipped / Total] 95 / 0 / 4 / 99:  95%|█████████▌| 95/100 [06:02<00:19,  3.81s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[14 (100%)]] --> [[16 (70%)]]

56ac ac23 2368 68ba ba11 11da dade de56 5659 5980 8010 1049 4988 885f 5f26 2600 0000 0001 0101 [[0108]] [[080a]] 0a8e 8ea5 a5ef ef6f 6f7c 7c34 3460 6047 475c 5c6d 6d92 92b3 b384 8444 4489 89e3 e3bc bc85 8507 0763 6352 522c 2c46 46df df35 354e 4e6f 6f34 3488 8859 59d9 d986 865f 5f1d 1d54 543a 3aee eec7 c7dd dd0f 0fe5 e5ea ea51

56ac ac23 2368 68ba ba11 11da dade de56 5659 5980 8010 1049 4988 885f 5f26 2600 0000 0001 0101 [[01d8]] [[d80a]] 0a8e 8ea5 a5ef ef6f 6f7c 7c34 3460 6047 475c 5c6d 6d92 92b3 b384 8444 4489 89e3 e3bc bc85 8507 0763 6352 522c 2c46 46df df35 354e 4e6f 6f34 3488 8859 59d9 d986 865f 5f1d 1d54 543a 3aee eec7 c7dd dd0f 0fe5 e5ea ea51




[Succeeded / Failed / Skipped / Total] 96 / 0 / 4 / 100:  96%|█████████▌| 96/100 [06:04<00:15,  3.79s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[6 (100%)]] --> [[19 (100%)]]

[[ce7e]] [[7eea]] ea4b 4b3a 3a06 0616 16a1 a123 23c4 c450 5018 [[18ff]] [[ffff]] ffca cad0 d000 0000 0017 1703 0301 0100 0020 2079 79e6 e60a 0abf bf86 8698 9818 1805 0557 5765 6507 07c0 c000 005b 5b5d 5d91 9195 95f9 f996 9677 7742 42c1 c10f 0f3a 3a8d 8d85 85d0 d04a 4a4e 4e3a 3aa0 a0f9

[[ce66]] [[66ea]] ea4b 4b3a 3a06 0616 16a1 a123 23c4 c450 5018 [[1801]] [[01ff]] ffca cad0 d000 0000 0017 1703 0301 0100 0020 2079 79e6 e60a 0abf bf86 8698 9818 1805 0557 5765 6507 07c0 c000 005b 5b5d 5d91 9195 95f9 f996 9677 7742 42c1 c10f 0f3a 3a8d 8d85 85d0 d04a 4a4e 4e3a 3aa0 a0f9



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 96     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:           

# 6. Display attack results

In [6]:
# Display the attack results and the differences
logger = CSVLogger(color_method="html")
for result in attack_results:
    logger.log_attack_result(result)
    continue

display_html(HTML(logger.df[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,e240 4067 67c0 c032 32e9 e967 67bf bfe4 e4f4 f480 8018 1821 21f0 f077 77a0 a000 0000 0001 0101 0108 080a 0a06 06c5 c561 61bc bcf5 f573 73aa aada da01 0100 0059 597d 7d12 1266 667b 7b40 4053 5306 0647 4745 451e 1e65 654c 4c05 056a 6a50 5074 744a 4a7e 7e79 7901 0100 0000 0000 0000 0000 0000 0000 0000 0000 004a 4a9c 9cc1,e240 405b 5bc0 c032 32e9 e967 67bf bfe4 e4f4 f480 80d7 d721 21f0 f077 77a0 a01b 1b00 0001 0101 0108 0827 2706 06c5 c561 61bc bcf5 f573 73aa aada da01 0100 0059 597d 7d12 1266 667b 7bec ec53 5306 0671 7145 451e 1e65 654c 4c05 056a 6a50 5074 7452 527e 7e79 7901 0100 0000 0000 0000 0000 0000 0000 0000 0000 004a 4a9c 9cc1
1,ffdc dc10 1099 992a 2aea ea5a 5a71 71fe fe15 1550 5010 10fa fa61 614c 4c1d 1d00 0000 00da dafe fe86 864a 4ae2 e220 20cc cc3f 3fb0 b027 273d 3dd1 d1e2 e2ac acc8 c88b 8b48 48a6 a6e4 e459 59f7 f73a 3a06 06ad ad7d 7d52 52f1 f1f6 f665 6561 6196 9621 2122 22ae ae68 68be be2f 2f7a 7ade deb3 b30c 0cf5 f5e9 e9c5 c5dd ddf8 f865,ff60 6010 1099 992a 2aea ea5a 5a71 71fe fe15 1550 50ef effa fa61 614c 4c1d 1d00 0000 00da dafe fe86 864a 4ae2 e220 20cc cc3f 3fb0 b027 273d 3dd1 d1e2 e2ac acc8 c88b 8bfa faa6 a6e4 e459 59f7 f73a 3a06 06ad ad7d 7de9 e9f1 f1f6 f665 6561 6196 9621 2122 22ae ae68 68be be2f 2f7a 7ade deb3 b30c 0cf5 f5ae aec5 c5dd ddf8 f865
2,01bb bb92 9201 018a 8af2 f279 798d 8dbe beb0 b080 8018 185a 5a80 8031 313e 3e00 0000 0001 0101 0108 080a 0a05 0548 48db db09 09fc fcb6 b66f 6f65 6517 1703 0300 0005 0554 542a 2a42 42dd dd0e 0eb4 b43a 3ae8 e8a8 a83d 3d63 6302 022d 2d3c 3c91 911d 1d60 6094 944e 4eb2 b2e5 e53c 3c18 1806 06d3 d34d 4d2c 2cc4 c4df dfdb db55,01bb bb92 9201 018a 8af2 f279 798d 8dbe beb0 b080 8018 185a 5a80 8031 313e 3e00 0000 0001 01a5 a508 080a 0a1e 1e48 48db db09 09fc fcb6 b66f 6f65 6517 1703 03a1 a105 0554 542a 2a42 42dd dd0e 0eb4 b43a 3ae8 e8a8 a83d 3d63 6302 022d 2d3c 3c91 911d 1d60 6094 944e 4eb2 b2e5 e53c 3c18 1806 06d3 d34d 4d2c 2cc4 c4df dfdb db55
3,2787 87ec ecfd fdf9 f973 73eb eb9f 9f27 2708 0880 8018 185a 5a80 80c8 c8ae ae00 0000 0001 0101 0108 080a 0a1e 1eb4 b4e0 e01e 1e0d 0d3d 3d58 58a5 a517 1703 0300 0005 0578 78a0 a043 432a 2a1d 1dce cea2 a216 16cc ccce ce4b 4b0e 0e02 0232 3231 31d1 d1e7 e7d5 d50c 0ca7 a70c 0cc8 c81d 1d89 8967 671f 1f1c 1ceb ebd8 d846 46d2,2787 87ec ecfd fdf9 f973 73eb eb9f 9f27 2708 0880 8018 185a 5a80 80c8 c8ae ae00 0000 0001 0101 0108 080a 0a05 05b4 b4e0 e01e 1e0d 0d3d 3d58 58a5 a517 1703 0300 0005 0578 78a0 a043 432a 2a1d 1dce cea2 a216 16cc ccce ce4b 4b0e 0e02 0232 3231 31d1 d1e7 e7d5 d50c 0ca7 a70c 0cc8 c81d 1d89 8967 671f 1f1c 1ceb ebd8 d846 46d2
4,0050 5086 86c4 c40d 0df3 f366 6660 6018 1802 0250 5018 18fa faf0 f08c 8c92 9200 0000 0047 4745 4554 5420 202f 2f78 7868 6864 6431 314f 4f42 4259 5938 3835 3564 644b 4b64 6458 5848 4845 4558 5877 7741 4144 4473 7377 7753 537a 7a79 796b 6b67 6730 305a 5a25 2532 3262 6245 4530 3047 474e 4e61 6136 3642 4236 3666 6672 726f,0050 5086 86c4 c40d 0df3 f366 6660 6018 1802 022d 2d18 18fa faf0 f08c 8c92 9200 0000 0047 4745 4554 5420 202f 2f78 7868 6864 6431 314f 4f42 4259 5938 3835 3564 644b 4b64 6458 5848 4845 4558 5877 7741 4144 4473 7377 7753 537a 7a79 796b 6b67 6730 305a 5a25 2532 3262 6245 4530 3047 474e 4e61 6136 3642 4236 3666 6672 726f
5,f266 66d7 d7ea ea3c 3cef ef57 57c9 c9d9 d914 1480 8018 1849 4988 88a6 a612 1200 0000 0001 0101 0108 080a 0a21 2158 582e 2e81 8102 029a 9aa0 a0c2 c216 1603 0300 0000 0037 3702 0200 0000 0033 3303 0300 0057 5794 9449 490d 0dc3 c320 203e 3e49 49c3 c358 583e 3e53 5343 4353 533e 3e4c 4cc3 c320 203e 3ed3 d343 4300 003e 3ed9,f266 66d7 d7ea ea3c 3cef ef57 57c9 c9d9 d914 1480 8018 1849 4988 88a6 a612 1200 0000 0001 0101 0108 080a 0a16 1658 582e 2e81 8102 029a 9aa0 a0c2 c216 1603 0300 0000 0037 37fb fb00 0000 0033 3303 0300 0057 5794 9449 490d 0dc3 c320 203e 3e49 49c3 c358 583e 3e53 5343 4353 533e 3e4c 4cc3 c320 203e 3ed3 d343 4300 003e 3ed9
6,ffdc dce6 e6d3 d3cc cc27 27a7 a7e0 e050 5098 9850 5010 10fa fa97 974b 4bab ab00 0000 00de de3c 3cc2 c2b4 b425 2568 6